# 问题2

## 模型声明

In [1]:
import numpy as np
import pandas as pd

class FixPrice:
    df1 = None       # 表一
    df2 = None       # 表二
    a = 0            # 系数
    b = 0            # 系数
    X1 = None        # 任务坐标
    X2 = None        # 会员坐标
    w_limit = None   # 任务吸引度阈值
    w = None         # 任务对会员的吸引度
    
    def __init__(self, a = 0.01175, b = 0.000164):
        df1 = pd.read_excel('dataset/附件一：已结束项目任务数据.xls')
        df2 = pd.read_excel('dataset/附件二：会员信息数据-分列.xlsx')
        self.df1 = df1
        self.df2 =df2
        self.a = a
        self.b = b
        
        C_before = np.array(df1['任务执行情况'])
        P_before = np.array(df1['任务标价'])
        
        X1 = np.array(df1[['任务gps 纬度','任务gps经度']])
        X2 = np.array(df2[['会员位置纬度','会员位置经度']])
        
        self.X1 = X1
        self.X2 = X2
        
        # 计算阈值
        w_before = np.zeros((X1.shape[0],X2.shape[0]))
        for i in range(X1.shape[0]):
            for j in range(X2.shape[0]):
                l = ((X1[i,0] - X2[j,0])**2 + (X1[i,1] - X2[j,1])**2)*111**2
                w_before[i,j] = np.sqrt(a/l + b * P_before[i]**2)
        self.w_limit = np.zeros(X1.shape[0])
        for i in range(X1.shape[0]):
            if C_before[i] == 1:
                self.w_limit[i] = np.min(w_before[i])
            else:
                self.w_limit[i] = np.max(w_before[i])
        print(self.w_limit)
                
    def makePrice(self,P):     # 根据定价求吸引度
        # 计算吸引度,距离单位为km
        self.w = np.zeros((self.X1.shape[0],self.X2.shape[0]))
        for i in range(self.X1.shape[0]):
            for j in range(self.X2.shape[0]):
                l = ((self.X1[i,0] - self.X2[j,0])**2 + (self.X1[i,1] - self.X2[j,1])**2)*111**2
                self.w[i,j] = np.sqrt(self.a/l + self.b * P[i]**2)
                
    def match(self):           # 匹配
        
        member_limit = np.array(self.df2['预订任务限额'].astype('float'))
        group = self.df2.groupby('预订任务开始时间')

        # 每个时间点有多少会员
        group_count = group.count()['会员编号']
        group_cumsum = np.array(group_count).cumsum()
        group_cumsum

        is_order = np.zeros(self.X1.shape[0])                    # 用于标记任务是否被预定
        C = np.zeros(self.X1.shape[0])                           # 用于标记任务是否完成
        for k in range(len(group_count)):                   # 遍历各时间点
            for j in range(group_count[k]):                 # 遍历各会员
                member_index = group_cumsum[k]-group_count[k]+j    # 当前会员的索引
                current_w = pd.Series(self.w[:,member_index])
                current_w = current_w.sort_values(ascending=False) # 任务按吸引度从大到小排
                work_index = current_w.index                       # 任务按吸引度排序后，应记录索引
                current_w.index = range(self.X1.shape[0])
                for i in range(self.X1.shape[0]):                # 遍历各任务

                    if member_limit[member_index] == 0:          # 配额为0，跳出循环
                        break
                    if current_w[i] > self.w_limit[work_index[i]] and is_order[work_index[i]] == 0: # 吸引力大于阈值,且该任务未被预定
                        C[work_index[i]] = 1                                                  # 任务i被完成
                        is_order[work_index[i]] = 1                                           # 任务i已被预定
                        member_limit[member_index] = member_limit[member_index] - 1                              # 会员配额减少
        return C

## 模型调用
将定价的区间逐步缩小

In [2]:
model = FixPrice()
P_all = np.array([[]]) # P_all保存每次的定价
C_all = []             # C_all保存每次完成的任务数
index = []             # index保存每次的定价范围
for i in range(15):
    index.append("[{},{}]".format(55+i,91-i))        # index保存每次的定价范围
    rand = np.random.seed(10)
    P = np.random.uniform(55+i,91-i,(1,835)).ravel() # 随机给出定价
    if i == 0:                                       # P_all保存每次的定价
        P_all = P
    else:    
        P_all = np.row_stack((P_all,P))
    model.makePrice(P)            # 修改定价
    c = model.match()             # 再次匹配计算
    C_all.append(c.sum())                            # C_all保存每次完成的任务数

[ 0.84819328  0.90991764  0.83880928  0.96533248  0.90402058  0.99332576
  0.83880928  0.85721637  1.12784935  0.8452124   1.06088948  0.86887458
  0.83880928  0.8452124   0.8452124   0.8452124   0.83880928  0.89595226
  0.83880928  0.8452124   0.8452124   0.83240615  0.86059259  0.88422214
  0.87047245  0.8452124   0.83880928  0.85161552  0.84848734  0.8754725
  0.90011198  0.8923659   0.83880928  0.8452124   1.46344639  0.83880928
  0.90967371  0.97099351  0.85813422  0.92520946  1.03287938  0.85161552
  0.88599943  0.96379423  0.93953128  0.87617912  0.86590579  1.01126521
  1.02745579  0.91549763  0.87425113  0.88685767  0.90329832  0.92330161
  0.88511036  0.93779827  1.04734696  0.88602792  0.97796739  0.983284
  0.8789715   0.88624259  0.97099351  0.96995448  0.90071787  0.86975154
  1.04680179  0.85281549  0.93535191  0.89385627  0.91666397  1.15605905
  0.85816116  0.87504011  0.87680919  0.92451351  0.83880928  0.85161552
  0.91049498  0.87799017  0.89643739  0.92832011  0.97

## 写入文件

In [3]:
df_ = pd.DataFrame(P_all)
df_.index = index
df_["完成率"] = np.array(C_all)/835
# df_.to_excel('不同定价对应结果1.xlsx')
df_

,0,1,2,3,4,5,6,7,8,9,...,826,827,828,829,830,831,832,833,834,完成率
"[55,91]",82.767543,55.747070,77.811336,81.956940,72.946252,63.092679,62.130263,82.379106,61.087990,58.180233,...,67.617815,70.451055,83.895338,63.714511,55.119962,68.991628,88.177353,60.494460,81.999801,0.664671
"[56,90]",82.224902,56.705566,77.544040,81.459332,72.949238,63.643086,62.734137,81.858044,61.749768,59.003554,...,67.916825,70.592663,83.290042,64.230372,56.113297,69.214316,87.334166,61.189212,81.499812,0.671856
"[57,89]",81.682261,57.664062,77.276744,80.961724,72.952224,64.193493,63.338012,81.336983,62.411547,59.826874,...,68.215836,70.734271,82.684745,64.746232,57.106633,69.437003,86.490980,61.883964,80.999823,0.679042
"[58,88]",81.139619,58.622558,77.009447,80.464116,72.955210,64.743899,63.941886,80.815921,63.073325,60.650194,...,68.514846,70.875879,82.079449,65.262093,58.099968,69.659690,85.647794,62.578716,80.499834,0.691018
"[59,87]",80.596978,59.581055,76.742151,79.966509,72.958196,65.294306,64.545760,80.294860,63.735103,61.473515,...,68.813856,71.017487,81.474152,65.777953,59.093304,69.882378,84.804608,63.273469,79.999845,0.698204
"[60,86]",80.054337,60.539551,76.474854,79.468901,72.961182,65.844713,65.149634,79.773799,64.396882,62.296835,...,69.112866,71.159096,80.868855,66.293814,60.086639,70.105065,83.961421,63.968221,79.499856,0.705389
"[61,85]",79.511695,61.498047,76.207558,78.971293,72.964168,66.395119,65.753509,79.252737,65.058660,63.120156,...,69.411877,71.300704,80.263559,66.809674,61.079975,70.327752,83.118235,64.662973,78.999867,0.711377
"[62,84]",78.969054,62.456543,75.940261,78.473685,72.967154,66.945526,66.357383,78.731676,65.720438,63.943476,...,69.710887,71.442312,79.658262,67.325535,62.073310,70.550440,82.275049,65.357725,78.499878,0.729341
"[63,83]",78.426413,63.415039,75.672965,77.976078,72.970140,67.495933,66.961257,78.210614,66.382217,64.766796,...,70.009897,71.583920,79.052966,67.841395,63.066646,70.773127,81.431863,66.052478,77.999889,0.735329
"[64,82]",77.883772,64.373535,75.405668,77.478470,72.973126,68.046340,67.565132,77.689553,67.043995,65.590117,...,70.308908,71.725528,78.447669,68.357256,64.059981,70.995814,80.588676,66.747230,77.499900,0.762874
